The purpose of this code is to sample predictor variables at our training locations

In [1]:
import os
import ee
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
import time
num_seed=30
random.seed(num_seed)

In [2]:
ee.Initialize()


In [3]:
#Define functions for mapping MapBiomas and simplifying the legend
coverage_palette =  ['ffffff', '129912', '1f4423', '006400', '00ff00', '687537', '76a5af', '29eee4', 
                     '77a605', '935132', 'bbfcac', '45c2a5', 'b8af4f', 'f1c232', 'ffffb2', 'ffd966', 
                     'f6b26b', 'f99f40', 'e974ed', 'd5a6bd', 'c27ba0', 'fff3bf', 'ea9999', 'dd7e6b', 
                     'aa0000', 'ff99ff', '0000ff', 'd5d5e5', 'dd497f', 'b2ae7c', 'af2a2a', '8a2be2', 
                     '968c46', '0000ff', '4fd3ff']


simple_palette = ['129912','BBFCAC','FFFFB2','EA9999','0000FF','D5D5E5']
statesViz = {'min': 0, 'max': 34, 'palette': coverage_palette};
simpleStatesViz = {'min': 1, 'max': 6, 'palette': simple_palette};

change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 0, 'max': 1, 'palette': change_detection_palette};

#Load in mapbiomas
mapbiomas_states=ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1')
states_mask = mapbiomas_states.mask()

#Define function to convert hierarchical legend to simplest form
def simplify_legend(bandName):
    simplify = mapbiomas_states.expression(
        '(b0 >=1)  && (b0<10) ? 1 :'+
        '((b0>=10) && (b0<14)) || (b0==32) || (b0==29) ? 2 :'+
        '((b0>=18) && (b0<22)) || ((b0>=14)&&(b0<16)) ? 3 :'+
        '((b0>=22) && (b0<26)) || (b0==30) ? 4 :'+
        '(b0==26) || (b0==33) || (b0==31) ? 5 : 6', 
        {
          'b0': mapbiomas_states.select([bandName])
        })
    simplify = simplify.select(['constant'],[bandName])
    return simplify

#Bands we are interested in
bandList = ['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 
             'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 
             'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 
             'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 
             'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 
             'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 
             'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 
             'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 
             'classification_2017', 'classification_2018']
bandsEEList = ee.List(bandList) 
states_simple = ee.ImageCollection(bandsEEList.map(simplify_legend)).toBands()
states_simple = states_simple.select(states_simple.bandNames(),bandsEEList)
states_simple = states_simple.updateMask(states_mask)
states_simple = states_simple.set(ee.Dictionary({'min_value':1,'max_value':5}))

#Map
Map1 = geemap.Map(center=[-9,-51], zoom=4)
Map1.addLayer(mapbiomas_states.select('classification_2018'),statesViz,name='Original MapBiomas')
Map1.addLayer(states_simple.select('classification_2018'),simpleStatesViz,name='Simplified MapBiomas')
display(Map1)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [4]:
#Save 30 meter projection
projection_30m = mapbiomas_states.projection()

In [5]:
#Convert long band names to short band names
intBandNames = ['1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', 
             '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',  '2005', '2006', '2007', '2008', 
             '2009', '2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018']

int_states_simple = states_simple.select(bandList,intBandNames)

def convert_to_image_collection(bandName):
    return int_states_simple.select([bandName])

int_states_simple_collection = ee.ImageCollection(int_states_simple.bandNames().map(convert_to_image_collection))


In [6]:
#Define functions for finding different types of change in the MapBiomas series
def lc_one_change(bandName):
    '''
    Determines if there was one change occurance from year i to year i+1. Returns an image with values:
    1 if state(i) != state(i+1)
    0 if state(i) == state(i+1)
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    return int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))

def lc_no_change(bandName):
    '''
    Determines if there was one change occurance from year i to year i+1. Returns an image with values:
    1 if state(i) != state(i+1)
    0 if state(i) == state(i+1)
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    return int_states_simple.select([currentYearString]).eq(int_states_simple.select([nextYearString]))


def lc_reverse(bandName):
    '''
    Determines if change that occured from i to i+1 reversed back to state i in i+2
    1 if state(i) != state(i+1) and state(i) == state(i+2)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    
    returnback = int_states_simple.select([currentYearString]).eq(int_states_simple.select([nextNextYearString]))
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    lc_reversed = returnback.bitwise_and(changed)
    return lc_reversed

def lc_change_to_another(bandName):
    '''
    Determines if change that occured from i to i+1 reversed back to state i in i+2
    1 if state(i) != state(i+1) and state(i) != state(i+2) and state(i+1) != state(i+2)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    changed_again = int_states_simple.select([nextYearString]).neq(int_states_simple.select([nextNextYearString]))
    not_reversed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextNextYearString]))

    lc_changed_to_another = changed.bitwise_and(changed_again.bitwise_and(not_reversed))
    return lc_changed_to_another


def lc_consistent_change_one_year(bandName):
    '''
    Determines if change that occured from i to i+1 stayed in i+2
    1 if state(i) != state(i+1) and state(i+1) == state(i+2)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    stayed = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextYearString]))
    lc_consistently_changed = changed.bitwise_and(stayed)
    return lc_consistently_changed

def lc_consistent_change_two_years(bandName):
    '''
    Determines if change that occured from i to i+1 stayed in i+2 and i+3
    1 if state(i) != state(i+1) and state(i+1) == state(i+2) and state(i+1) == state(i+3)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    nextNextNextYear = currentYear.add(ee.Number(3))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    nextNextNextYearString = ee.List(intBandNames).get(nextNextNextYear)
    
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    stayed_one_year = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextYearString]))
    stayed_two_year = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextNextYearString]))
    lc_consistently_changed = changed.bitwise_and(stayed_one_year.bitwise_and(stayed_two_year))
    return lc_consistently_changed

def lc_year_after(bandName):
    '''
    Determines if change that occured from i to i+1 stayed in i+2 and i+3
    1 if state(i) != state(i+1) and state(i+1) == state(i+2) and state(i+1) == state(i+3)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    nextNextNextYear = currentYear.add(ee.Number(3))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    
    nextYearValue = int_states_simple.select([nextYearString])
    return nextYearValue

def convert_classification_to_multiband(image):
    '''
    Function to convert image with one band for land cover classifications to image with multiple bands,
    one band for each land cover class with indicator function of LC, right now just need to rename the bands...
    '''
    min_value = image.get('min_value')
    max_value = image.get('max_value')
    bands_array = ee.List.sequence(start=min_value, end=max_value)
    band_collection = ee.ImageCollection(bands_array.map(lambda x: image.eq(ee.Image.constant(x))))
    band_image = band_collection.toBands()
    return band_image


In [7]:
#Apply functions to MapBiomas image series
years_for_one_change = np.arange(0,len(intBandNames)-1).tolist()#
years_for_two_change = np.arange(0,len(intBandNames)-2).tolist()#
years_for_three_change = np.arange(0,len(intBandNames)-3).tolist()#

states_lc_no_change = ee.ImageCollection(ee.List(years_for_one_change).map(lc_no_change)).toBands()
states_lc_no_change = states_lc_no_change.select(states_lc_no_change.bandNames(),intBandNames[0:len(intBandNames)-1])

states_lc_one_change = ee.ImageCollection(ee.List(years_for_one_change).map(lc_one_change)).toBands()
states_lc_one_change = states_lc_one_change.select(states_lc_one_change.bandNames(),intBandNames[0:len(intBandNames)-1])

states_lc_reverse = ee.ImageCollection(ee.List(years_for_two_change).map(lc_reverse)).toBands()
states_lc_reverse = states_lc_reverse.select(states_lc_reverse.bandNames(),intBandNames[0:len(intBandNames)-2])

states_lc_change_to_another = ee.ImageCollection(ee.List(years_for_two_change).map(lc_change_to_another)).toBands()
states_lc_change_to_another = states_lc_change_to_another.select(states_lc_change_to_another.bandNames(),intBandNames[0:len(intBandNames)-2])

states_lc_consistent_change_one_year = ee.ImageCollection(ee.List(years_for_two_change).map(lc_consistent_change_one_year)).toBands()
states_lc_consistent_change_one_year = states_lc_consistent_change_one_year.select(states_lc_consistent_change_one_year.bandNames(),intBandNames[0:len(intBandNames)-2])

states_lc_consistent_change_two_years = ee.ImageCollection(ee.List(years_for_three_change).map(lc_consistent_change_two_years)).toBands()
states_lc_consistent_change_two_years = states_lc_consistent_change_two_years.select(states_lc_consistent_change_two_years.bandNames(),intBandNames[0:len(intBandNames)-3])

states_class_band_coll = ee.ImageCollection(int_states_simple_collection.map(convert_classification_to_multiband))


states_lc_year_after = ee.ImageCollection(ee.List(years_for_one_change).map(lc_year_after)).toBands()
states_lc_year_after = states_lc_year_after.select(states_lc_year_after.bandNames(),intBandNames[0:len(intBandNames)-1])


In [8]:
#Map states_class_band_coll
Map2 = geemap.Map(center=[-9,-51], zoom=4)
Map2.addLayer(int_states_simple.select('1985'),simpleStatesViz,name='Simple MapBiomas')
Map2.addLayer(states_class_band_coll.first(),name='States True False')
display(Map2)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [9]:

#Function to convert feature collection to pandas dataframe
def get_dataframe_from_feature_collection(feature_collection, property_names):
    df = pd.DataFrame()
    for property_name in property_names:
        property_values = feature_collection.aggregate_array(property_name).getInfo()
        df[property_name] = property_values
    return df

#Function to convert pandas dataframe to feature collection
def convert_points_df_to_feature_collection(df,projection='EPSG:4326',lat_name='latitude',lon_name='longitude'):
    feature_collection_list = []
    for i,row in df.iterrows():
        geometry = ee.Geometry.Point([row[lon_name],row[lat_name]])#,projection)
        row_dict = row.to_dict()
        row_feature = ee.Feature(geometry,row_dict)
        feature_collection_list.append(row_feature)
    return ee.FeatureCollection(feature_collection_list)

#Function to convert pandas dataframe to feature collection
def convert_point_df_to_feature(series,projection='EPSG:4326',lat_name='latitude',lon_name='longitude'):
    geometry = ee.Geometry.Point([series[lon_name],series[lat_name]])#,projection)
    row_dict = series.to_dict()
    row_feature = ee.Feature(geometry,row_dict)
    return row_feature

def getSampleImageData(image, sampleBandPoints):
    sampleImageData = image.reduceRegions(
            collection=sampleBandPoints,
            reducer=ee.Reducer.first()
            )
    return sampleImageData.aggregate_array('first')

def getSampleImageDataFromPoint(image, sampleBandPoint):
    sampleImageData = image.reduceRegion(
            geometry=sampleBandPoint.geometry(),
            reducer=ee.Reducer.first()
            )
    return sampleImageData.get(image.bandNames().get(0))


In [10]:
#Read in training data
training_points_url = 'https://raw.githubusercontent.com/kristinelister/WRI-NGS-DynamicWorld/master/MapBiomas_ChangeDetection/TrainingPoints/Sample_Points_1000_wchange.csv'
training_points = pd.read_csv(training_points_url)
training_points_fc = convert_points_df_to_feature_collection(training_points)


In [11]:
#Map training points

#Find pixels that had at least one year of consistent change, add 1 so that 0 can be the mask/no data value
consistent_change_occurred = states_lc_consistent_change_two_years.reduce(ee.Reducer.max()).add(1)
#Update mask
consistent_change_occurred_masked = consistent_change_occurred.updateMask(states_lc_one_change.reduce(ee.Reducer.max()))
#Rename band from "max" from reducer to "consistent_change"
consistent_change_occurred_masked = consistent_change_occurred_masked.select(['max'],['consistent_change'])
#Consistent change raster is now coded:
# 0 = no data, change did not occur in this pixel in any year
# 1 = no consistent change in this pixel in any year
# 2 = at least one year of consistent change occurred in this pixel
                                                                          
#Define color palettes and map
consistent_change_detection_palette = ['df07b5','0741df']
consistentChangeDetectionViz = {'min': 1, 'max': 2, 'palette': consistent_change_detection_palette};

Map2 = geemap.Map(center=[-9,-51], zoom=4)
Map2.addLayer(consistent_change_occurred_masked,consistentChangeDetectionViz,name='Consistent Change')
Map2.addLayer(training_points_fc,name='Sampled Points')
#Pink shows there was not consistent change, blue shows consistent change
display(Map2)
#Pink is no change
#Blue is change

#Consistent change raster is now coded:
# 0 = no data, change did not occur in this pixel in any year
# 1 = no consistent change in this pixel in any year (PINK)
# 2 = at least one year of consistent change occurred in this pixel (BLUE)



Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [12]:
#Define kernel for neighborhood
kernel = ee.Kernel.fixed(3,3,
                         [[1,1,1],
                          [1,0,1],
                          [1,1,1]]
                          ,1,1)

#Other kernel options commented below:
#kernel = ee.Kernel.gaussian(radius=1000, units='meters', sigma=1000)

#kernel = ee.Kernel.circle(radius=17, units='pixels')
#kernel = ee.Kernel.square(radius=1.5, units='pixels')


In [13]:
#Get neighborhood information on pixels
states_lc_no_change_neighbors = states_lc_no_change.convolve(kernel)
states_lc_one_change_neighbors = states_lc_one_change.convolve(kernel)
states_lc_reverse_neighbors = states_lc_reverse.convolve(kernel)
states_lc_change_to_another_neighbors = states_lc_change_to_another.convolve(kernel)
states_lc_consistent_change_one_year_neighbors = states_lc_consistent_change_one_year.convolve(kernel)
states_lc_consistent_change_two_years_neighbors = states_lc_consistent_change_two_years.convolve(kernel)


In [14]:
#Define list of images and the corresponding names (which will be used as column names) to sample for covariates
image_list = [int_states_simple,states_lc_year_after,states_lc_no_change_neighbors,states_lc_one_change_neighbors,states_lc_reverse_neighbors,
             states_lc_change_to_another_neighbors,states_lc_consistent_change_one_year_neighbors,
             states_lc_consistent_change_two_years_neighbors]
image_name_list = ['int_states_simple','states_lc_year_after','states_lc_no_change_neighbors','states_lc_one_change_neighbors','states_lc_reverse_neighbors',
             'states_lc_change_to_another_neighbors','states_lc_consistent_change_one_year_neighbors',
                  'states_lc_consistent_change_two_years_neighbors']

In [15]:

#Sample covariates from image list defined above at our training point locations
training_points_wcovar = training_points.copy()

#Iterate through each row of the dataframe
for i,row in training_points_wcovar.iterrows():
    #Convert row to EE feature
    row_fc = convert_point_df_to_feature(row)
    #Iterate through image list
    for zippy in zip(image_list,image_name_list):
        image = zippy[0]
        image_name = zippy[1]
        #Reproject image to 30 meter resolution
        image = image.reproject(projection_30m).select(str(int(row['year'])))
        #Get value of image at point location
        output = getSampleImageDataFromPoint(image, row_fc).getInfo()
        #Save to dataframe
        training_points_wcovar.at[i,image_name] = output
        training_points_wcovar.to_csv('/Users/kristine/Downloads/NewSampledPoints_wcovariates.csv',index=False)

        
            
display(training_points_wcovar)


,consistent_change,latitude,longitude,year,int_states_simple,states_lc_year_after,states_lc_no_change_neighbors,states_lc_one_change_neighbors,states_lc_reverse_neighbors,states_lc_change_to_another_neighbors,states_lc_consistent_change_one_year_neighbors,states_lc_consistent_change_two_years_neighbors
0,0,-20.323530,-41.074882,2014,1.0,3.0,8.0,0.0,0.0,0.0,0.0,0.0
1,0,-9.748832,-63.887330,2015,1.0,3.0,0.0,8.0,0.0,0.0,8.0,2.0
2,0,-3.562314,-60.710258,2015,1.0,3.0,3.0,5.0,0.0,0.0,5.0,0.0
3,0,-3.482274,-42.444723,2014,3.0,1.0,6.0,2.0,0.0,0.0,2.0,1.0
4,0,-18.723002,-56.919278,2015,2.0,1.0,6.0,2.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,-3.548839,-45.688091,2012,3.0,1.0,6.0,2.0,0.0,0.0,2.0,2.0
1996,1,-5.488122,-35.278323,1990,4.0,3.0,7.0,1.0,0.0,0.0,1.0,1.0
1997,1,-14.634499,-41.844828,2000,3.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0
1998,1,-3.396036,-39.927644,1992,1.0,3.0,3.0,5.0,0.0,0.0,5.0,5.0


In [16]:
#Export to CSV on local computer, which will be uploaded to Github
training_points_wcovar.to_csv('/Users/kristine/Downloads/NewSampledPoints_wcovariates.csv',index=False)
